
-------------------------------------------------------------------------------
GEOLOGICAL TIMELAPSE GENERATOR (Google Earth Engine)
-------------------------------------------------------------------------------
A Python tool for geologists to generate high-resolution, scientific-grade timelapses of depositional environments (deltas, inlets, canyon erosion) using 40 years of Landsat satellite data.<br>

Author: Brian Willis (programmed to a large extent by Gemini AI)<br>
AI programming support: Gemini 3 pro<br>
Contact: Willis77019@gmail.com<br>
License:  MIT License (unrestricted: Use at your own risk)<br>

# Overview
This script uses the Google Earth Engine (GEE) API to process satellite imagery in the cloud. It is designed to:<br>
•	Remove Clouds: Uses a "surgical" pixel QA mask to remove clouds and shadows.<br>
•	Fix Defects: Automatically skips broken Landsat 7 data (scan line failure) to ensure clean images.<br>
•	Enhance Geology: Offers a "Geology Mode" (False Color SWIR/NIR/Red) to highlight moisture, vegetation, and lithology differences.<br>
•	Export 4K-Ready Frames: Generates a sequence of GeoTIFFs suitable for professional video editing (DaVinci Resolve, Premiere).<br>
________________________________________
# Prerequisites

Before running this code, you must have a Google Earth Engine account.<br>
A. Sign Up for Earth Engine<br>
&nbsp;&nbsp;&nbsp;&nbsp;o	Go to the Earth Engine Sign-Up Page.<br>
&nbsp;&nbsp;&nbsp;&nbsp;o	Log in with your Google Account.<br>
&nbsp;&nbsp;&nbsp;&nbsp;o	Complete the form.<br>
&nbsp;&nbsp;&nbsp;&nbsp;o	Organization: "Personal" or your university/company.<br>
&nbsp;&nbsp;&nbsp;&nbsp;o	Project Use: "Geological research and visualization."<br>
&nbsp;&nbsp;&nbsp;&nbsp;o	Note: Approval is usually instantaneous but can sometimes take a few days.<br>
<br>
2. Create a Cloud Project<br>
Google requires a "Cloud Project" to track your requests. This is free for research/educational use.<br>
&nbsp;&nbsp;&nbsp;&nbsp;o	Visit the Earth Engine Code Editor.<br>
&nbsp;&nbsp;&nbsp;&nbsp;o	If prompted, click "Register a new Cloud Project".<br>
&nbsp;&nbsp;&nbsp;&nbsp;o	Give it a simple name (e.g., geology-timelapse-project).<br>
&nbsp;&nbsp;&nbsp;&nbsp;o	Important: Copy this Project ID (e.g., geology-timelapse-project). You will need to paste it into the script.<br>
<br>
3. Install the EarthEngine Library in your Python environment <br>
(if you use Colab this will already be installed)<br>
&nbsp;&nbsp;&nbsp;&nbsp;o Open your terminal (or command prompt) and run:<br>
&nbsp;&nbsp;&nbsp;&nbsp;pip install earthengine-api<br>
One More Local python envionment "Gotcha": Authentication<br>
When you run this locally for the first time after installing, the ee.Authenticate() line will likely open a browser window.
It will ask you to log in. It will give you a code (or ask you to create a local token). Follow the instructions in the terminal. You only have to do this once on your computer. After that, the script will run automatically.

________________________________________
# How to Run (No Installation Required)
The easiest way to run this tool is using Google Colab, which runs Python in your browser without installing anything on your computer.<br>
1.	Click Here to Open a New Colab Notebook.<br>
2.	Copy the script code below.<br>
3.	Paste it into the main code cell.<br>
4.	Configure the Settings at the top of the script:<br>
&nbsp;&nbsp;&nbsp;&nbsp;o	Update PROJECT_ID with your own project name.<br>
&nbsp;&nbsp;&nbsp;&nbsp;o	Change CENTER_LAT / CENTER_LON to your target site.<br>
&nbsp;&nbsp;&nbsp;&nbsp;o	Set VISUALIZATION_MODE to 'GEOLOGY' or 'TRUE_COLOR'.<br>
5.	Click the "Play" (Run) button.<br>
6.	Authorize: A pop-up will ask for permission to access your Google Drive. Check the boxes and click "Continue."<br>
7.	Wait: The script will submit export tasks to Google's servers.<br>
8.	Download: After ~10-15 minutes, check your Google Drive for a folder named EarthEngine_Timelapse_Exports. It will contain the high-res images.<br>

This can also be run locally on JupyterLab of other python environment by installing the EarthEngine Library (see above)
____________________________________________
# Notes
One way to look for areas with interesting time lapse imagery is to look at the google earth video<br>
&nbsp;&nbsp;&nbsp;&nbsp;https://earthengine.google.com/timelapse/ <br>
&nbsp;&nbsp;&nbsp;&nbsp;Follow the " Head to Google Earth to explore more" link.<br>
&nbsp;&nbsp;&nbsp;&nbsp;In Google Earth right click to "copy coodinates" (Enter these coodinates inot the script below)<br>
    
Tips for Post-Processing<br>
&nbsp;&nbsp;&nbsp;&nbsp;o Smoothing: Use AI Frame Interpolation (like Flowframes or DaVinci Resolve Optical Flow) to smooth the jump between the annual images.<br>
&nbsp;&nbsp;&nbsp;&nbsp;o Gap Year: The year 2012 is skipped because Landsat 5 was decommissioned and Landsat 8 had not launched. Interpolation software handles this 2-year jump easily.<br>

In [ ]:
import ee

# ==========================================
#      SECTION 1: USER CONFIGURATION
#      (Edit these values for each new site)
# ==========================================

# 1. VISUALIZATION MODE
#    'GEOLOGY'    = False Color (SWIR/NIR/Red) -> Best for ridges, water limits, sediment.
#    'TRUE_COLOR' = Standard Photography (Red/Green/Blue) -> Best for general audiences.
VISUALIZATION_MODE = 'GEOLOGY' 

# 2. LOCATION (Decimal Degrees)
#    Example: North Topsail Beach, NC
CENTER_LAT = 34.63087
CENTER_LON = -77.153

# 3. ZOOM / BOX SIZE (in Degrees)
#    0.10 = ~11 km (Inlet), 0.25 = ~27 km (Delta), 0.50 = ~55 km (Regional)
BOX_SIZE = 0.15

# 4. DATE RANGE
#    (Note: The script AUTOMATICALLY skips 2012 due to data gaps)
START_YEAR = 1985
END_YEAR = 2023

# 5. SEASONAL FILTER (Month Numbers)
#    Use the Dry/Summer season to minimize clouds and vegetation noise.
#    North America Summer: 6 (June) to 9 (Sept)
START_MONTH = 6
END_MONTH = 9

# 6. PROJECT ID
#    IMPORTANT: Replace this with YOUR Earth Engine Project ID.
PROJECT_ID = 'INSERT_YOUR_PROJECT_ID_HERE'

# ==========================================
#      SECTION 2: SYSTEM SETUP
#      (Do not edit below this line)
# ==========================================

try:
    ee.Initialize(project=PROJECT_ID)
except Exception:
    ee.Authenticate()
    ee.Initialize(project=PROJECT_ID)

roi = ee.Geometry.Rectangle([
    CENTER_LON - BOX_SIZE/2, CENTER_LAT - BOX_SIZE/2,
    CENTER_LON + BOX_SIZE/2, CENTER_LAT + BOX_SIZE/2
])

print(f"Target: {CENTER_LAT}, {CENTER_LON} | Mode: {VISUALIZATION_MODE}")

# ==========================================
#      SECTION 3: PROCESSING FUNCTIONS
# ==========================================

def mask_clouds(image):
    """
    Surgically removes clouds/shadows using the QA_PIXEL band.
    """
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(1<<1).eq(0).And(qa.bitwiseAnd(1<<3).eq(0).And(qa.bitwiseAnd(1<<4).eq(0)))
    return image.updateMask(mask)

def apply_scale_factors(image):
    """
    Converts raw integer data to Reflectance (0.0 - 1.0) to prevent white screens.
    """
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    return image.addBands(optical_bands, None, True)

def get_annual_composite(year):
    """
    Generates a single, cloud-free image for the year.
    CRITICAL: Skips Landsat 7 entirely to prevent 'Striping'.
    """
    start = ee.Date.fromYMD(year, START_MONTH, 1)
    end = ee.Date.fromYMD(year, END_MONTH, 30)

    # Define Bands based on User Choice
    if VISUALIZATION_MODE == 'GEOLOGY':
        # False Color (SWIR1, NIR, Red)
        l8_bands = ['SR_B6', 'SR_B5', 'SR_B4'] # L8/9 Source
        l5_bands = ['SR_B5', 'SR_B4', 'SR_B3'] # L5 Source
        out_names = ['Red_Ch', 'Green_Ch', 'Blue_Ch'] # Standardized Output
    else:
        # True Color (Red, Green, Blue)
        l8_bands = ['SR_B4', 'SR_B3', 'SR_B2']
        l5_bands = ['SR_B3', 'SR_B2', 'SR_B1']
        out_names = ['Red_Ch', 'Green_Ch', 'Blue_Ch']

    # A. LANDSAT 8 & 9 (Modern)
    l89 = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
           .merge(ee.ImageCollection("LANDSAT/LC09/C02/T1_L2"))
           .filterDate(start, end).filterBounds(roi)
           .map(mask_clouds).map(apply_scale_factors)
           .map(lambda img: img.select(l8_bands).rename(out_names)))

    # B. LANDSAT 5 (Historic) - NO LANDSAT 7 INCLUDED
    l5 = (ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
           .filterDate(start, end).filterBounds(roi)
           .map(mask_clouds).map(apply_scale_factors)
           .map(lambda img: img.select(l5_bands).rename(out_names)))

    return l89.merge(l5).median().clip(roi).set('year', year)

# ==========================================
#      SECTION 4: EXPORT EXECUTION
# ==========================================

# Visualization Parameters
if VISUALIZATION_MODE == 'GEOLOGY':
    vis_params = {'bands': ['Red_Ch', 'Green_Ch', 'Blue_Ch'], 'min': 0.0, 'max': 0.45, 'gamma': 1.4}
else:
    vis_params = {'bands': ['Red_Ch', 'Green_Ch', 'Blue_Ch'], 'min': 0.0, 'max': 0.20, 'gamma': 1.2}

print("Starting Processing...")
# Generate list of years, EXCLUDING 2012 (Data Gap)
years = [y for y in range(START_YEAR, END_YEAR + 1) if y != 2012]

for year in years:
    try:
        img = get_annual_composite(year)
        rgb_img = img.visualize(**vis_params)
        
        # Filename includes location, mode, and year
        safe_mode = "GEO" if VISUALIZATION_MODE == 'GEOLOGY' else "RGB"
        fname = f"Site_{safe_mode}_{str(CENTER_LAT).replace('.','p')}_{year}"
        
        task = ee.batch.Export.image.toDrive(
            image=rgb_img, description=fname, folder='EarthEngine_Timelapse_Exports',
            region=roi, scale=30, fileFormat='GeoTIFF'
        )
        task.start()
        print(f"Submitted task: {year}")
    except Exception as e:
        print(f"Error on {year}: {e}")

print("\nAll tasks submitted! Check Drive folder 'EarthEngine_Timelapse_Exports'.")